In [1]:
!pip install dash
!pip install ipython-sql
!pip install psycopg2-binary
!pip install pandas
!pip install numpy
!pip install numpy matplotlib
!pip install folium
!pip install geojson
!pip install jobspy
!pip install dash-bootstrap-components

In [2]:
import numpy as np
import pandas as pd
import psycopg2
import pandas.io.sql as sqlio
import matplotlib.pyplot as plt
import pylab as pl
import folium
import json
import os
from folium import plugins
%matplotlib inline

import plotly.graph_objects as go
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import plotly.express as px
import dash_bootstrap_components as dbc
import dash_core_components as dcc

<ipython-input-2-284909b32e1d>:15: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-2-284909b32e1d>:16: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [3]:
with open('Data_to_Otomoto.json') as file:
    Data_to_Otomoto = json.loads(file.read())
car_brand = Data_to_Otomoto["car_brand"]


In [4]:
psql = psycopg2.connect(host='192.168.10.163', port='5432', database='Otomoto', user='barto', password='biznes')

In [5]:
cur = psql.cursor()
sql_otomoto = "SELECT * FROM otomoto_10;"
dat_otomoto = sqlio.read_sql_query(sql_otomoto, psql)
conn = None

In [6]:
car_loc = dat_otomoto[["cena","marka_pojazdu","przebieg","rok_produkcji"]]

In [7]:
a = 0
for index, row in car_loc.iterrows():
    if (np.isnan(car_loc.at[index,'przebieg']) or np.isnan(car_loc.at[index,'rok_produkcji']) 
    or np.isnan(car_loc.at[index,'cena'])):
        car_loc = car_loc.drop([index])
        a += 1
print("Drop rows where Nan from table otomoto: ", a)

Drop rows where Nan from table otomoto:  23


In [8]:
car_loc = car_loc.reset_index(drop=True)

In [9]:
car_loc.head()

,cena,marka_pojazdu,przebieg,rok_produkcji
0,93500.0,Abarth,1.0,2021.0
1,69900.0,Abarth,25191.0,2018.0
2,61900.0,Abarth,30631.0,2017.0
3,67500.0,Abarth,18993.0,2019.0
4,37900.0,Abarth,86220.0,2010.0


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=car_loc[rok_produkcji], y=car_loc[cena],
                    mode='lines',
                    name='lines'))
fig.show()

In [10]:
car_loc.shape

(11882, 4)

In [ ]:
app = dash.Dash(__name__,external_stylesheets=[dbc.themes.LITERA])

app.layout = html.Div(
html.H1('Cars for sell in Otomoto',style={'textAlign': 'center','front-size' :50}))



if __name__ == '__main__':
    app.run_server(host='0.0.0.0')

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.17.0.2:8050/ (Press CTRL+C to quit)
192.168.10.24 - - [05/Oct/2021 20:04:38] "GET / HTTP/1.1" 200 -
192.168.10.24 - - [05/Oct/2021 20:04:39] "GET /assets/stylesheet.css?m=1633374912.958587 HTTP/1.1" 304 -
192.168.10.24 - - [05/Oct/2021 20:04:39] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
192.168.10.24 - - [05/Oct/2021 20:04:39] "GET /_dash-layout HTTP/1.1" 200 -
192.168.10.24 - - [05/Oct/2021 20:04:39] "GET /_dash-dependencies HTTP/1.1" 200 -
192.168.10.24 - - [05/Oct/2021 20:04:39] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
192.168.10.151 - - [05/Oct/2021 20:04:47] "GET / HTTP/1.1" 200 -
192.168.10.151 - - [05/Oct/2021 20:04:47] "GET /assets/stylesheet.css?m=1633374912.958587 HTTP/1.1" 304 -
192.168.10.151 - - [05/Oct/2021 20:04:47] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
192.168.10.151 - - [05/Oct/2021 20:04:47] "GET /_dash-dependencies HTTP/1.1" 200 -
192.168.10.151 - - [05/Oct/2021 20:04:47] "GET /_dash-layout HTTP/1.1" 200 -

192.168.10.151 - - [05/Oct/2021 20:05:48] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
192.168.10.151 - - [05/Oct/2021 20:05:48] "GET /_dash-component-suites/dash/html/dash_html_components.min.js.map HTTP/1.1" 200 -
192.168.10.151 - - [05/Oct/2021 20:05:48] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.js.map HTTP/1.1" 200 -
192.168.10.151 - - [05/Oct/2021 20:05:48] "GET /_dash-layout HTTP/1.1" 200 -
192.168.10.151 - - [05/Oct/2021 20:05:48] "GET /_dash-component-suites/dash/dcc/dash_core_components.js.map HTTP/1.1" 200 -
192.168.10.151 - - [05/Oct/2021 20:05:48] "GET /_dash-dependencies HTTP/1.1" 200 -
192.168.10.151 - - [05/Oct/2021 20:05:48] "GET /_dash-component-suites/dash/dash_table/bundle.js.map HTTP/1.1" 200 -
192.168.10.151 - - [05/Oct/2021 20:05:50] "GET / HTTP/1.1" 200 -
192.168.10.151 - - [05/Oct/2021 20:05:50] "GET /assets/stylesheet.css?m=1633374912.958587 HTTP/1.1" 304 -
192.168.10.151 - - [05/Oct/2021 20:05:50] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 